Code: fJYR928ga

Lecturer: Prof. Elvis Ndah

In [ ]:
import string
import numpy as np
import pandas as pd
from random import randint
import nltk

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split


from pickle import dump, load

In [ ]:
import os 

os.chdir('/content/drive/MyDrive/NLP_files/Assignment2')
filelist = os.listdir(os.getcwd())
filelist

['Things_Fall_Apart.txt',
 'Arrow_of_God.txt',
 'Anthills_of_the_Savannah.txt',
 'A_Man_of_the_People.txt']

In [ ]:
#doc = []
s = ''
for dir in filelist:
  file = open(dir, 'r', encoding='latin-1')
  #doc.append( file.read() )
  s += str(file.read())
  file.close()

In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
    """Clean text"""
    doc = doc.replace('--', ' ')
    tokens = doc.split() # split into tokens by white space
    table = str.maketrans('', '', string.punctuation) # remove punctuation
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()] # remove non-alphabetic tokens
    tokens = [word.lower() for word in tokens]

    return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [ ]:
tokens = clean_doc(s)
print(tokens[:20])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['things', 'fall', 'apart', 'chinua', 'achebe', 'first', 'published', 'in', 'one', 'of', 'the', 'first', 'african', 'novels', 'written', 'in', 'english', 'to', 'receive', 'global']
Total Tokens: 275070
Unique Tokens: 14817


In [ ]:
# organize into sequences of tokens
length = 50 + 1 
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    sequences.append(line)
    
print('Total Sequences: %d' % len(sequences))

Total Sequences: 275019


In [ ]:
save_doc(sequences, 'text_sequence.txt')

In [ ]:
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [ ]:
doc = load_doc('text_sequence.txt')
lines = doc.split('\n')

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

14818


In [ ]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

seq_length = X.shape[1]

In [ ]:
X.shape, y.shape

((275019, 50), (275019, 14818))

In [ ]:
type(X)

numpy.ndarray

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            740900    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 14818)             1496618   
                                                                 
Total params: 2,388,418
Trainable params: 2,388,418
Non-trainable params: 0
_________________________________________________________________
None
